# Captura de Paquetes con Scapy

In [1]:
from scapy.all import sniff, wrpcap

# Función para capturar paquetes en tiempo real y guardarlos en un archivo .pcap
def capture_and_save_packets(packet_count=10, filename='captured_packets.pcap'):
    packets = sniff(filter="tcp port 80 or tcp port 443", count=packet_count)
    wrpcap(filename, packets)
    print(f"Se han capturado y guardado {packet_count} paquetes en {filename}")

# Capturar y guardar paquetes en un archivo .pcap
capture_and_save_packets(packet_count=10, filename='captured_packets.pcap')


Se han capturado y guardado 10 paquetes en captured_packets.pcap


**Paquetes Capturados con Scapy**:
    
- **`from scapy.all import sniff, wrpcap`**: Importa las funciones necesarias de la biblioteca Scapy para capturar y guardar paquetes.

- **`capture_and_save_packets`**: Define una función para capturar paquetes en tiempo real usando un filtro para tráfico HTTP y HTTPS y guardarlos en un archivo `.pcap`.
    
- **`capture_and_save_packets(packet_count=10, filename='captured_packets.pcap')`**: Captura y guarda 10 paquetes en un archivo llamado `captured_packets.pcap`.

# Análisis de Paquetes con PyShark

In [2]:
import pyshark
import pandas as pd
import nest_asyncio

# Permitir que el bucle de eventos de asyncio funcione en Jupyter Notebook
nest_asyncio.apply()

# Función para analizar paquetes capturados utilizando PyShark
def analyze_packets(pcap_file):
    capture = pyshark.FileCapture(pcap_file)
    packet_data = []
    for packet in capture:
        try:
            protocol = packet.transport_layer
            src_addr = packet.ip.src
            src_port = packet[packet.transport_layer].srcport
            dst_addr = packet.ip.dst
            dst_port = packet[packet.transport_layer].dstport
            packet_data.append({
                'protocol': protocol,
                'src_addr': src_addr,
                'src_port': src_port,
                'dst_addr': dst_addr,
                'dst_port': dst_port
            })
            print(f"{protocol} Paquete: {src_addr}:{src_port} -> {dst_addr}:{dst_port}")
        except AttributeError:
            # Ignorar paquetes sin capa de transporte
            pass
    return pd.DataFrame(packet_data)

# Análisis de paquetes capturados
pcap_file = 'captured_packets.pcap'  # Nombre del archivo pcap
df_packets = analyze_packets(pcap_file)

# Mostrar los datos analizados en Jupyter Notebook
display(df_packets.head())


TCP Paquete: 162.159.133.234:443 -> 192.168.1.11:38048
TCP Paquete: 192.168.1.11:38048 -> 162.159.133.234:443
TCP Paquete: 140.82.113.25:443 -> 192.168.1.11:48930
TCP Paquete: 192.168.1.11:48930 -> 140.82.113.25:443
TCP Paquete: 140.82.113.25:443 -> 192.168.1.11:48930
TCP Paquete: 192.168.1.11:38048 -> 162.159.133.234:443
TCP Paquete: 162.159.133.234:443 -> 192.168.1.11:38048
TCP Paquete: 162.159.133.234:443 -> 192.168.1.11:38048
TCP Paquete: 192.168.1.11:38048 -> 162.159.133.234:443
TCP Paquete: 192.168.1.11:51912 -> 162.247.243.29:443


,protocol,src_addr,src_port,dst_addr,dst_port
0,TCP,162.159.133.234,443,192.168.1.11,38048
1,TCP,192.168.1.11,38048,162.159.133.234,443
2,TCP,140.82.113.25,443,192.168.1.11,48930
3,TCP,192.168.1.11,48930,140.82.113.25,443
4,TCP,140.82.113.25,443,192.168.1.11,48930


 **Análisis de Paquetes con PyShark**:
    
- **`import pyshark, pandas as pd, nest_asyncio`**: Importa las bibliotecas necesarias para el análisis de paquetes y manipulación de datos.

- **`nest_asyncio.apply()`**: Permite que el bucle de eventos de asyncio funcione en Jupyter Notebook.

- **`analyze_packets(pcap_file)`**: Define una función para analizar paquetes capturados utilizando PyShark y extraer información relevante (protocolo, direcciones IP y puertos).

- **`df_packets = analyze_packets(pcap_file)`**: Analiza el archivo `.pcap` capturado y guarda los datos en un DataFrame de Pandas.

- **`display(df_packets.head())`**: Muestra los primeros cinco registros del DataFrame en Jupyter Notebook.

# Almacenamiento de Paquetes en SQLite

In [3]:
import sqlite3

# Crear base de datos y tabla
conn = sqlite3.connect('network_traffic.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS traffic
             (protocol TEXT, src_addr TEXT, src_port INTEGER, dst_addr TEXT, dst_port INTEGER)''')

# Función para almacenar paquetes en la base de datos
def store_packet(packet):
    try:
        protocol = packet['protocol']
        src_addr = packet['src_addr']
        src_port = packet['src_port']
        dst_addr = packet['dst_addr']
        dst_port = packet['dst_port']
        c.execute("INSERT INTO traffic (protocol, src_addr, src_port, dst_addr, dst_port) VALUES (?, ?, ?, ?, ?)",
                  (protocol, src_addr, src_port, dst_addr, dst_port))
        conn.commit()
        print(f"Stored packet: {protocol} {src_addr}:{src_port} -> {dst_addr}:{dst_port}")
    except AttributeError as e:
        print(f"Packet skipped: {e}")

# Almacenar los datos analizados
for index, row in df_packets.iterrows():
    store_packet(row)

# Verificar los datos almacenados
df = pd.read_sql_query("SELECT * FROM traffic", conn)
print(df)

# Mostrar los datos en Jupyter Notebook
display(df.head())

# Cerrar la conexión a la base de datos
conn.close()


Stored packet: TCP 162.159.133.234:443 -> 192.168.1.11:38048
Stored packet: TCP 192.168.1.11:38048 -> 162.159.133.234:443
Stored packet: TCP 140.82.113.25:443 -> 192.168.1.11:48930
Stored packet: TCP 192.168.1.11:48930 -> 140.82.113.25:443
Stored packet: TCP 140.82.113.25:443 -> 192.168.1.11:48930
Stored packet: TCP 192.168.1.11:38048 -> 162.159.133.234:443
Stored packet: TCP 162.159.133.234:443 -> 192.168.1.11:38048
Stored packet: TCP 162.159.133.234:443 -> 192.168.1.11:38048
Stored packet: TCP 192.168.1.11:38048 -> 162.159.133.234:443
Stored packet: TCP 192.168.1.11:51912 -> 162.247.243.29:443
  protocol         src_addr  src_port         dst_addr  dst_port
0      TCP  162.159.133.234       443     192.168.1.11     38048
1      TCP     192.168.1.11     38048  162.159.133.234       443
2      TCP    140.82.113.25       443     192.168.1.11     48930
3      TCP     192.168.1.11     48930    140.82.113.25       443
4      TCP    140.82.113.25       443     192.168.1.11     48930
5      

,protocol,src_addr,src_port,dst_addr,dst_port
0,TCP,162.159.133.234,443,192.168.1.11,38048
1,TCP,192.168.1.11,38048,162.159.133.234,443
2,TCP,140.82.113.25,443,192.168.1.11,48930
3,TCP,192.168.1.11,48930,140.82.113.25,443
4,TCP,140.82.113.25,443,192.168.1.11,48930


**Almacenamiento de Paquetes con SQLite**:

- **`import sqlite3`**: Importa la biblioteca SQLite3 para el manejo de la base de datos.

- **`conn = sqlite3.connect('network_traffic.db')`**: Crea una conexión a la base de datos SQLite llamada `network_traffic.db`.

- **`c.execute('''CREATE TABLE IF NOT EXISTS traffic...''')`**: Crea una tabla llamada `traffic` si no existe, con columnas para el protocolo, direcciones IP y puertos.

- **`store_packet(packet)`**: Define una función para almacenar un paquete en la base de datos.

- **`for index, row in df_packets.iterrows()`**: Itera sobre cada fila del DataFrame y almacena los datos en la base de datos.

- **`df = pd.read_sql_query("SELECT * FROM traffic", conn)`**: Recupera todos los registros de la tabla `traffic` y los guarda en un DataFrame.

- **`display(df.head())`**: Muestra los primeros cinco registros del DataFrame en Jupyter Notebook.

- **`conn.close()`**: Cierra la conexión a la base de datos.